<a href="https://colab.research.google.com/github/GuilhermePelegrina/Mackenzie/blob/main/Aulas/2s2024/TIC/Aula_05_Aspectos_adicionais_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/logo_mackenzie.png'>


# **Aspectos adicionais sobre treinamento e teste**

Nesta aula discutiremos alguns aspectos adicionais sobre as etapas de treinamento e teste. Mais precisamente, serão abordadas técnicas usadas na definição dos conjuntos de treinamento e teste para criar o modelo computacional e avaliar seu desempenho. Além disso, veremos outras formas de avaliar o modelo obtido, explorando, por exemplo, o conceito de equidade em aprendizado de máquina.

Para esta aula, usaremos a base de dados [COMPAS Recidivism Risk](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing), da ProPublica. Nesse [link](https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Datasets/data_compas_projC1_train.csv), há a opção por baixar o conjunto de dados. A base de dados COMPAS contém diversos atributos descrevendo pessoas detidas por crimes. Para cada pessoa, há também a classificação como um indivíduo potencialemnte reincidente ou não.

In [ ]:
# Lendo os dados

import pandas as pd

dados = pd.read_csv('https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Datasets/data_compas_projC1_train.csv')
dados.head()

## Como separar dados em treinamento e teste?

Vimos nas aulas passadas métodos usados em problemas de classificação. Nos problemas abordados em sala, após separamos os dados em treinamento e teste, criamos o modelo computacional se baseando nos dados de treinamento e avaliamos o desempenho do mesmo nos dados de teste. Essa técnica é chamada de **Hold-out**. No entanto, ela não fornece uma medida de robustez do modelo. Em outras palavras, não temos informação da sensibilidade do modelo para diferentes combinações de dados de treinamento e teste. Para avaliar esse aspecto, há duas abordagens mais usuais, uma baseada em **simulações do Hold-out** e outra na **Validação Cruzada**.

### Simulações do Hold-out

Para avaliar a robustez do modelo para diferentes *splits* dos dados, podemos gerar diversos cenários em que a separação dos dados entre treinamento e teste partiu de sementes diferentes. Para cada cenário, criamos o modelo e avaliamos seu desempenho. Note que, como diversas configurações foram consideradas, podemos extrair informações como o desempenho médio, desvio padrão, e até selecionar o modelo cujo desempenho foi o melhor.

Veja nos códigos abaixo como gerar essas simulações e extrair informações adicionais sobre a construção do modelo computacional. Usaremos como exemplo a regressão logística.

In [ ]:
# Carregando o modelo
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

# Preparando os dados
dados.dropna(inplace=True) # Removendo NaN
X = dados.drop('score_risk', axis=1)
y = dados['score_risk']

# Convertendo variáveis categóricas em dummies
dummies = pd.get_dummies(X)
X = pd.concat([X, dummies],axis=1)
X.drop(columns=['sex', 'age_cat','race','c_charge_degree'], inplace=True)

In [ ]:
# Simulações aleatórias
from sklearn import metrics
from sklearn.model_selection import train_test_split

nSimul = 10 # Número de simulações
acuracia = [] # Lista vazia para guardar as acurácias

melhor_modelo_acuracia = 0
acuracia_atual = 0
melhor_acuracia = 0

for ii in range(nSimul):

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify = y) # Split aleatório
  logreg.fit(X_train,y_train) # Treinando o modelo
  y_pred = logreg.predict(X_test) # Fazendo predições
  cnf_matrix = metrics.confusion_matrix(y_test, y_pred) # Matriz de confusão
  acuracia_atual = (cnf_matrix[0][0] + cnf_matrix[1][1])/len(y_pred) # Acurácia do modelo atual

  if len(acuracia) == 0:
    melhor_modelo_acuracia = logreg
    melhor_acuracia = acuracia_atual
  else:
    if acuracia_atual > melhor_acuracia:
      melhor_modelo_acuracia = logreg
      melhor_acuracia = acuracia_atual

  acuracia.append(acuracia_atual) # Atualiza a lista de acurácias

print(acuracia)
print(melhor_acuracia)

In [ ]:
# Simulações com sementes definidas

sementes = [1,3,59,59,21] # Semente definidas
acuracia = [] # Lista vazia para guardar as acurácias

melhor_modelo_acuracia = 0
acuracia_atual = 0
melhor_acuracia = 0

for ii in sementes:

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=ii, stratify = y) # Split aleatório
  logreg.fit(X_train,y_train) # Treinando o modelo
  y_pred = logreg.predict(X_test) # Fazendo predições
  cnf_matrix = metrics.confusion_matrix(y_test, y_pred) # Matriz de confusão
  acuracia_atual = (cnf_matrix[0][0] + cnf_matrix[1][1])/len(y_pred) # Acurácia do modelo atual

  if len(acuracia) == 0:
    melhor_modelo_acuracia = logreg
    melhor_acuracia = acuracia_atual
  else:
    if acuracia_atual > melhor_acuracia:
      melhor_modelo_acuracia = logreg
      melhor_acuracia = acuracia_atual

  acuracia.append(acuracia_atual) # Atualiza a lista de acurácias

print(acuracia)
print(melhor_acuracia)

### Validação cruzada de *k* pastas (*k*-fold cross-validation)

A validação cruzada é uma outra forma de avaliar o classificador construído (e selecionar o que teve melhor resultado). A ideia consiste em dividir os dados em *k* partes iguais (ex: em *k=5* partes, sendo que cada parte contém 20% dos dados) e avaliar o desempenho dos classificadores alterando quais são os dados de teste. Ou seja, na primeira iteração, 1 primeira das *k* pastas é usada no teste e o restante no treinamento. Na segunda iteração, a segunda pasta é usada no teste e o restante no treinamento. E assim por diante. Com isso, garantimos que não há repetição de dados de teste nas diferentes avaliações do modelo adotado.

Veja como fica nos códigos abaixo:

In [ ]:
from sklearn.model_selection import KFold # Importando a biblioteca do k-fold

nFold = 5 # Definindo o número de pastas
k_fold = KFold(nFold,shuffle=True) # Definindo a estrutura do k-fold

acuracia = [] # Lista vazia para guardar as acurácias

melhor_modelo_acuracia = 0
acuracia_atual = 0
melhor_acuracia = 0

for ii, (train, test) in enumerate(k_fold.split(X, y, groups=y)): # Nesse loop, as listas train e test indicam os índices das amostras em cada conjunto de dados
  X_train, X_test, y_train, y_test = X.iloc[train,:], X.iloc[test,:], y.iloc[train], y.iloc[test] # Define os dados de treinamento e de teste

  logreg.fit(X_train,y_train) # Treinando o modelo
  y_pred = logreg.predict(X_test) # Fazendo predições
  cnf_matrix = metrics.confusion_matrix(y_test, y_pred) # Matriz de confusão
  acuracia_atual = (cnf_matrix[0][0] + cnf_matrix[1][1])/len(y_pred) # Acurácia do modelo atual

  if len(acuracia) == 0:
    melhor_modelo_acuracia = logreg
    melhor_acuracia = acuracia_atual
  else:
    if acuracia_atual > melhor_acuracia:
      melhor_modelo_acuracia = logreg
      melhor_acuracia = acuracia_atual

  acuracia.append(acuracia_atual) # Atualiza a lista de acurácias

print(acuracia)
print(melhor_acuracia)

## Métricas de avaliação de um classificador

Até o momento, vimos apenas a acurácia como medida de desempenho de um classificador. A acurácia (ou *accuracy score*) é uma métrica simples que mede a proporção de previsões corretas feitas pelo modelo. É a razão entre o número de previsões corretas e o número total de previsões, ou seja, a porcentagem de acertos. Veja abaixo um código que calcula diretamente a acurácia:

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report # Import de várias ferramentas para avaliar classificadores

# Vamos criar um novo modelo de regressão logística, com seed = 1 no train_test_split e estratificando em y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, stratify = y)
logreg.fit(X_train,y_train) # Treinando o modelo
y_pred = logreg.predict(X_test) # Fazendo predições

# Calcular a accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score:", accuracy)

A acurácia, por si só, não é suficiente para verificar a capacidade preditiva de um modelo de classificação em muitos casos, especialmente quando os dados são desbalanceados ou quando as classes têm diferentes custos associados a erros de classificação. Existem várias razões pelas quais a acurácia isoladamente pode ser enganosa ou inadequada:

1. **Desbalanceamento de Classes**: Em problemas em que as classes são
desequilibradas, ou seja, uma classe tem muito mais exemplos do que a outra, a acurácia pode ser enganosa. O modelo pode atingir uma alta acurácia simplesmente prevendo a classe majoritária em quase todos os casos, enquanto falha em identificar a classe minoritária. Nesse caso, a acurácia não reflete a capacidade do modelo de identificar corretamente as amostras da classe minoritária.

2. **Custo dos Erros**: Em muitos cenários, cometer um tipo específico de erro pode ser mais crítico do que outro. Por exemplo, em diagnósticos médicos, um falso negativo (não diagnosticar uma doença quando ela está presente) pode ter consequências mais graves do que um falso positivo (diagnosticar erroneamente uma doença). A acurácia não leva em consideração o custo associado a diferentes tipos de erros.

3. **Contexto do Problema**: A escolha de métricas de avaliação deve ser guiada pelo contexto do problema. O que é mais importante: minimizar falsos positivos, minimizar falsos negativos, encontrar um equilíbrio entre ambos ou otimizar para outra métrica específica? A acurácia não leva em consideração as nuances do problema.

4. **Métricas Personalizadas**: Em alguns casos, métricas personalizadas podem ser mais relevantes do que a acurácia. Por exemplo, em aplicações de detecção de fraudes, você pode criar uma métrica personalizada que leve em consideração o impacto financeiro dos erros de classificação.

Portanto, a acurácia é uma métrica valiosa, mas não deve ser usada isoladamente. É importante considerar métricas adicionais que se alinhem com os objetivos e as peculiaridades do problema de classificação em questão, especialmente quando há desequilíbrio de classes ou custos desiguais associados a diferentes tipos de erros. Essas métricas proporcionarão uma compreensão mais completa do desempenho do modelo. A seguir vamos entender como a matriz de confusão pode ajudar neste processo.






## Relembrando: Matriz de confusão

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

cnf_matrix = confusion_matrix(y_test, y_pred, labels=logreg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix, display_labels=logreg.classes_)
disp.plot()

Portanto, a matriz de confusão fornece uma visão detalhada do desempenho do seu modelo em relação às duas classes e é frequentemente usada para calcular métricas como precisão, recall, F1-Score e outras métricas de avaliação.

Com base nesses valores, você pode avaliar o desempenho do seu modelo em relação a diferentes tipos de erros e acertos.Isso pode ser feito pelo classification report.

# Classification Report:

O relatório de classificação (classification report) fornece uma visão mais detalhada do desempenho do modelo, incluindo métricas como precisão, recall, pontuação F1 e suporte para cada classe.



## Como calcular ?

In [ ]:
# Gerar o relatório de classificação
report = classification_report(y_test, y_pred)
print(report)

##Como interpretar?
Vou explicar linha por linha o que o código do classification_report faz:

Classification_report: Esta é a função do scikit-learn que gera um relatório de classificação. Ele leva duas entradas obrigatórias: y_test (valores verdadeiros das classes dos dados que vc separou para treino) e y_pred (valores previstos pelo modelo).

**Precisão (Precision)**: A precisão é a proporção de previsões positivas corretas em relação ao total de previsões positivas. É calculada como: $$TP / (TP + FP)$$ onde TP são os verdadeiros positivos e FP são os falsos positivos.

**Recall (Revocação ou Sensibilidade)**: O recall é a proporção de previsões positivas corretas em relação ao total de amostras verdadeiramente positivas. É calculado como: $$TP / (TP + FN)$$ onde TP são os verdadeiros positivos e FN são os falsos negativos.

**F1-Score**: O F1-Score é a média harmônica entre precisão e recall e fornece uma única métrica que leva em consideração tanto falsos positivos quanto falsos negativos. É calculado como 2 $×$ (Precisão * Recall) / (Precisão + Recall).

**Suporte (Support)**: O suporte é o número de amostras verdadeiras de cada classe no conjunto de teste.

A saída do classification_report é uma string formatada que exibe essas métricas para cada classe no problema de classificação. A saída inclui as métricas para cada classe, bem como a média ponderada dessas métricas, levando em consideração o número de amostras em cada classe.

Vale ressaltar que a definição de qual classe é a positive ou qual é a negativa é relativo. Então, para essa análise, o importante é verificar a relação entre o real e o que foi predito. Não importando qual classe é a positiva e qual é a negativa. Podemos considerar a classe de maior interesse como positiva, por exemplo, e se basear nisso para guiar as análises.

# **Equidade em Aprendizado de Máquina**

Além de procurar por melhores desempenhos em modelos de aprendizado de máquina, há também preocupações éticas de seu uso. São frequentes os casos nos quais há disparidades éticas associadas a aspectos sensíveis da população, como gênero, raça, faixa etária, etc.

Com o intuito de eliminar tais disparidades, diversas regulações sobre o uso da Inteligência Artificial vem surgindo pelo mundo todo. Via de regra, a recomendação (ou, mais precisamente, a *obrigação*) é que os modelos de intelgência artificial / aprendizado de máquina sejam capazes de lidar com os problemas práticos mas que não gerem impactos negativos (dentre eles, os ligados às disparidades éticas) na população.

Para ilustrar esses cenários "injustos", vamos avaliar e comparar os desempenhos do classificador, nos dados COMPAS, para brancos e negros.

Nos comandos abaixo, há um exemplo com os dados de treinamento. No caso do modelo que foi desenvolvido para o projeto, avalie os desempenhos nos dados de teste.

In [ ]:
# Comparando a acurácia em relação aos brancos e negros

accuracy_brancos = accuracy_score(y_test[X_test['race_Caucasian'] == 1], y_pred[X_test['race_Caucasian'] == 1])
print("Acurácia para brancos:", accuracy_brancos)

accuracy_negros = accuracy_score(y_test[X_test['race_African-American'] == 1], y_pred[X_test['race_African-American'] == 1])
print("Acurácia para negros:", accuracy_negros)

In [ ]:
# Comparando o número de indivíduos erroneamente classificados como reincidentes, em relação aos brancos e negros

erro_brancos = len(y_test[(X_test['race_Caucasian'] == 1) & (y_test[y_test == -1]) & (y_test[y_pred == 1])])/len(y_test[X_test['race_Caucasian'] == 1])
print("Erro para brancos:", erro_brancos)

erro_negros = len(y_test[(X_test['race_African-American'] == 1) & (y_test[y_test == -1]) & (y_test[y_pred == 1])])/len(y_test[X_test['race_African-American'] == 1])
print("Erro para negros:", erro_negros)

print('Diferença:', erro_negros - erro_brancos)

In [ ]:
# Comparando o número de indivíduos erroneamente classificados como não-reincidentes, em relação aos brancos e negros

erro_brancos = len(y_test[(X_test['race_Caucasian'] == 1) & (y_test[y_test == 1]) & (y_test[y_pred == -1])])/len(y_test[X_test['race_Caucasian'] == 1])
print("Erro para brancos:", erro_brancos)

erro_negros = len(y_test[(X_test['race_African-American'] == 1) & (y_test[y_test == 1]) & (y_test[y_pred == -1])])/len(y_test[X_test['race_African-American'] == 1])
print("Erro para negros:", erro_negros)

print('Diferença:', erro_brancos - erro_negros)